In [1]:
import torch

#系统变量
batch_size=8
device=torch.device("cuda:0"if torch.cuda.is_available() else "cpu")
epochs=10

In [2]:
def load_image(path):
    import cv2
    import numpy as np
    import os

    data = []
    flag = []
    for fileName in os.listdir(path):
        if not fileName.endswith(".jpg"):
            continue

        #用opencv读取图片
        x=cv2.imread(path+"/"+fileName)

        #数值压缩到0-1之间
        x=torch.FloatTensor(np.array(x))/255

        #变形，使通道数在前面
        x=x.permute(2,0,1)

        #标签
        y=int(fileName[0])

        data.append(x)
        flag.append(y)

    return data,flag


In [3]:
#读取数据
data,flag=load_image("data\cifar10")

In [4]:
len(data),len(flag),data[0].shape

(60000, 60000, torch.Size([3, 32, 32]))

In [5]:
#定义数据集
class Dataset(torch.utils.data.Dataset):
    def __len__(self):
        return len(data)
    def __getitem__(self, item):
        return data[item],flag[item]

In [6]:
#建立数据集
dataset=Dataset()

In [7]:
#定义加载器
loader=torch.utils.data.DataLoader(dataset=dataset,
                                   batch_size=batch_size,
                                   shuffle=True,
                                   drop_last=True)

In [8]:
# 定义CNN模型
class CNNModule(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # 第一层CNN
        self.cnn1 = torch.nn.Conv2d(in_channels=3,
                                    out_channels=16,
                                    kernel_size=5,
                                    stride=2,
                                    padding=0)
        # 第二层CNN
        self.cnn2 = torch.nn.Conv2d(in_channels=16,
                                    out_channels=32,
                                    kernel_size=3,
                                    stride=1,
                                    padding=1)

        # 一层池化层
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2)

        # 第三层CNN
        self.cnn3 = torch.nn.Conv2d(in_channels=32,
                                    out_channels=128,
                                    kernel_size=7,
                                    stride=1,
                                    padding=0)

        # 激活函数
        self.relu = torch.nn.ReLU()

        # 全连接层
        self.fc = torch.nn.Linear(in_features=128, out_features=10)

    def forward(self, x):
        # 进入CNN1
        x = self.cnn1(x)
        x = self.relu(x)

        # CNN2
        x = self.cnn2(x)
        x = self.relu(x)

        # 池化层
        x = self.pool(x)

        # CNN3
        x = self.cnn3(x)
        x = self.relu(x)

        # 张量展平成一维
        x = x.flatten(start_dim=1)

        # 进入全连接层
        x = self.fc(x)
        return x

In [9]:
#
model=CNNModule()
model.to(device)

CNNModule(
  (cnn1): Conv2d(3, 16, kernel_size=(5, 5), stride=(2, 2))
  (cnn2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn3): Conv2d(32, 128, kernel_size=(7, 7), stride=(1, 1))
  (relu): ReLU()
  (fc): Linear(in_features=128, out_features=10, bias=True)
)

In [10]:
def train():
    optimizer= torch.optim.Adam(model.parameters(),lr=1e-3)
    loss_fu=torch.nn.CrossEntropyLoss()
    loss_fu.to(device)
    model.train()


    for epoch in range(epochs):
        for i,(x,y) in enumerate(loader):

            #数据移到GPU
            x=x.to(device)
            y=y.to(device)

            #forward
            out=model(x)
            loss=loss_fu(out,y)

            #backward
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if i% 2000==0:
                acc= (out.argmax(dim=1)==y).sum().item()/len(y)
                print(epoch,i,loss.item(),acc)

    torch.save(model,'cnn.model')


In [11]:
train()

0 0 2.306580066680908 0.25
0 2000 0.9257637858390808 0.625
0 4000 1.4971426725387573 0.5
0 6000 1.1215736865997314 0.5
1 0 1.01800537109375 0.625
1 2000 1.233528971672058 0.625
1 4000 0.7819304466247559 0.75
1 6000 1.517415165901184 0.5
2 0 0.9943893551826477 0.375
2 2000 1.5049068927764893 0.375
2 4000 0.9782319068908691 0.5
2 6000 2.0092337131500244 0.375
3 0 0.5837476849555969 0.75
3 2000 0.7204196453094482 0.625
3 4000 1.3456692695617676 0.625
3 6000 1.6329982280731201 0.375
4 0 1.0683062076568604 0.5
4 2000 0.7371273636817932 0.75
4 4000 0.2532336115837097 0.875
4 6000 0.38516801595687866 0.875
5 0 0.21899628639221191 1.0
5 2000 0.3681996762752533 0.875
5 4000 1.2307730913162231 0.5
5 6000 0.12378639727830887 1.0
6 0 0.6002645492553711 0.75
6 2000 0.954902172088623 0.75
6 4000 0.6272275447845459 0.625
6 6000 0.5577312707901001 0.75
7 0 0.9440266489982605 0.625
7 2000 0.7469991445541382 0.75
7 4000 1.3517078161239624 0.5
7 6000 1.004006266593933 0.75
8 0 0.4649518132209778 0.625
8 

In [12]:
#测试
@torch.no_grad()
def test():
    model=torch.load("cnn.model")
    
    model.to(device)

    model.eval()

    correct=0
    total=0
    for i in range(100):
        x,y=next(iter(loader))


        #GPU
        x=x.to(device)
        y=y.to(device)


        out=model(x).argmax(dim=1)

        correct+=(out==y).sum().item()
        total+=len(y)

    print(correct/total)


In [13]:
test()

0.86
